In [18]:
import pyomo.environ as pyo
from pyomo.environ import *
import idaes

In [42]:
d = 20
m = pyo.ConcreteModel()

m.PRODUCTS = pyo.Set(initialize  = ["U","V"])
m.x = pyo.Var(domain = NonNegativeReals)
m.y = pyo.Var(m.PRODUCTS, domain = NonNegativeReals)

@m.Constraint()
def demand_ub(m):
    return m.y["U"] <= 40
    
@m.Constraint()
def demand_lb(m):
    return m.y["U"] >= d

@m.Constraint()
def labor_A(m):
    return m.y["U"] + m.y["V"] <= 80

@m.Constraint()
def labor_B(m):
    return 2*m.y["U"] + m.y["V"] <= 100

@m.Constraint()
def production(m):
    return m.x >= 10*m.y["U"] + 9*m.y["V"]

@m.Expression()
def revenue(m):
    return 270*m.y["U"] + 210*m.y["V"]

@m.Expression()
def expense(m):
    return 130*m.y["U"] + 90*m.y["V"] + 10*m.x

@m.Objective(sense = pyo.maximize)
def profit(m):
    return m.revenue - m.expense

solver = pyo.SolverFactory('ipopt') # NL programs need ipopt : interior point optimization, probably overkill here
solver.solve(m)

print("U produced: ", pyo.value(m.y["U"]))
print("V produced: ", pyo.value(m.y["V"]))
print('profit: ', value(m.profit))

U produced:  20.000000199697926
V produced:  60.000000600272095
profit:  2600.0000260925735
